In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
tf.enable_eager_execution()
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [0]:
import os
# import tensorflow as tf
import xml.etree.ElementTree
import numpy as np
import pandas as pd 
import re
import matplotlib.pyplot as plt
from IPython.display import Image
from PIL import Image
from numpy import array
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Using TensorFlow backend.


In [0]:
print(tf.__version__)

1.15.2


In [0]:
## https://github.com/wisdal/diagnose-and-explain/blob/master/prepare_dataset.py

def extract_data():
    all_findings = []
    all_impressions = []
    all_img_names = []
    rids = []
    
    total_count = 0 # Count of reports available in the dataset
    no_image_count = 0 # Count of reports having no associated chest image
    no_impression_count = 0 # Count of reports having an empty "Impression" section
    no_findings_count = 0 # Count of reports having an empty "Findings" section

    # Storing impressions, findings and the image names in vectors
    for file in tqdm(os.listdir(annotation_folder)):
        total_count += 1
        file = os.path.abspath(annotation_folder) + '/' + file
        e = xml.etree.ElementTree.parse(file).getroot()

        rid = e.find('pmcId').get('id') # Report Id
        # We choose to ignore reports having no associated image
        image_id = e.find('parentImage')
        if image_id is None:
            no_image_count += 1
            continue

        image_id = image_id.get('id')
#         image_name = os.path.abspath('.') + '/' + image_id + '.png'
        image_name = image_folder + '/' + image_id + '.png'
        findings = ''
        impression = ''

        # Parsing "Impression" and "Findings"
        for element in e.findall('MedlineCitation/Article/Abstract/AbstractText'):
            if element.get('Label') == 'FINDINGS':
                findings = element.text
            if element.get('Label') == 'IMPRESSION':
                impression = element.text

        # Sanity check: Skip this report if it has an empty "Impression" section
        if findings is None:
            no_findings_count += 1
            #findings = 'No finding'
            continue
        if impression is None:
            no_impression_count += 1
            continue
        
        # Transforming findings and impressions into lists of sentences
        # https://stackoverflow.com/questions/21840389/python-regular-expression-remove-period-from-number-at-end-of-sentence
        findings = findings.replace("XXXX", "") #"XXXX" represents information anonymized
        findings=re.sub('((\d+)[\.])(?!([\d]+))','\g<2>',findings)
        findings = re.sub(" \d+", " ", findings)
#         sentences = findings.split('.')
        sentences = findings
#         del sentences[-1]
#         sentences = ['<start> ' + sentence + ' <end>' for sentence in sentences]
        sentences = ['<start> ' + sentences + ' <end>']
        findings = sentences

        impression = impression.replace("XXXX", "") #"XXXX" represents information anonymized
        impression=re.sub('((\d+)[\.])(?!([\d]+))','\g<2>',impression)
        impression = re.sub(" \d+", " ", impression)
#https://www.tutorialspoint.com/How-to-remove-specific-characters-from-a-string-in-Python
        impression=impression.replace("1", "")
#         sentences = impression.split('.')
#         del sentences[-1]
        sentences = impression
#         sentences = ['<start> ' + sentence + ' <end>' for sentence in sentences]
        sentences = ['<start> ' + sentences + ' <end>' ]
        impression = sentences

        #appending to vectors
        all_img_names.append(image_name)
        all_findings.append(findings)
        all_impressions.append(impression)
        rids.append(rid)

    print("Number of reports available:", total_count)
    print("Number of reports selected:", len(all_img_names))
    print("Number of reports not having images (skipped):", no_image_count)
    print("Number of reports with Impression section empty (skipped):", no_impression_count)
    print("Number of reports with Findings section empty:", no_findings_count)
    print("Total skipped:", no_image_count + no_impression_count + no_findings_count)

    return all_findings, all_impressions, all_img_names, rids

In [0]:
image_folder = '/content/drive/My Drive/Colab Notebooks/Medical data case study'
annotation_folder = '/content/drive/My Drive/Colab Notebooks/Medical data case study/ecgen-radiology'

In [0]:
all_findings, all_impressions, all_img_names, report_id=extract_data()

100%|██████████| 3955/3955 [00:09<00:00, 414.64it/s]

Number of reports available: 3955
Number of reports selected: 3331
Number of reports not having images (skipped): 104
Number of reports with Impression section empty (skipped): 6
Number of reports with Findings section empty: 514
Total skipped: 624


In [0]:
## Converting Lists to Dataframe
## https://stackoverflow.com/questions/30522724/take-multiple-lists-into-dataframe
df = pd.DataFrame(
    { 'report_id':report_id,
        'findings': all_findings,
     'impressions': all_impressions,
     'image_names': all_img_names
    })

In [0]:
df.head(10)

,report_id,findings,impressions,image_names
0,584,[<start> There are no focal areas of consolida...,[<start> No acute cardiopulmonary abnormality....,/content/drive/My Drive/Colab Notebooks/Medica...
1,580,[<start> The heart is normal in size. The medi...,[<start> No acute disease. <end>],/content/drive/My Drive/Colab Notebooks/Medica...
2,585,[<start> There are prominent epicardial fat pa...,[<start> No acute cardiopulmonary abnormality....,/content/drive/My Drive/Colab Notebooks/Medica...
3,581,[<start> Frontal and lateral views of the ches...,"[<start> No acute or active cardiac, pulmonary...",/content/drive/My Drive/Colab Notebooks/Medica...
4,586,[<start> The heart size is moderately enlarged...,[<start> Probable left midlung and left basila...,/content/drive/My Drive/Colab Notebooks/Medica...
5,582,[<start> Cardiomediastinal silhouette and pulm...,[<start> No acute cardiopulmonary findings. <e...,/content/drive/My Drive/Colab Notebooks/Medica...
6,589,[<start> Heart size normal. Lungs are clear. ...,[<start> Normal chest <end>],/content/drive/My Drive/Colab Notebooks/Medica...
7,587,[<start> The cardiomediastinal silhouette is w...,[<start> No interval change in the appearance...,/content/drive/My Drive/Colab Notebooks/Medica...
8,59,"[<start> The cardiac silhouette, mediastinum, ...",[<start> No acute abnormalities are seen. . <e...,/content/drive/My Drive/Colab Notebooks/Medica...
9,590,[<start> The cardiomediastinal silhouette is n...,[<start> Right middle lobe airspace disease ma...,/content/drive/My Drive/Colab Notebooks/Medica...


In [0]:
train_captions, img_name_vector = shuffle(all_impressions,
                                          all_img_names,
                                          random_state=1)


In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (512, 624))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-2].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [0]:
# tf.enable_eager_execution()

In [0]:
# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

In [0]:
1-2

-1

In [0]:
print(df['findings'][3])
print("-"*50)
print(df['impressions'][3])

In [0]:
Image(filename=df['image_names'][i], width=300, height=300) 

In [0]:
### https://pythonadventures.wordpress.com/2011/04/04/get-the-size-dimension-of-an-image/
# for i in range(10):
#     image_file = df['image_names'][i]
#     im = Image.open(image_file)
#     print(im.size)   # return value is a tuple, ex.: (1200, 800)

#### We would take size of image as (512,624)
for details uncomment the above snippet and run

In [0]:
def preprocess_image(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(512, 624))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [0]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [0]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)
# # model.add(Dense(4096))
# b=Dense(4096)(model)

In [0]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess_image(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once


In [0]:
start = time()
encoding_train = {}
for img in tqdm(df['image_names']):
    encoding_train[img] = encode(img)
print("Time taken in seconds =", time()-start)

In [0]:
encoding_train[img].shape

In [0]:
# Save the bottleneck train features to disk
with open("./encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

In [0]:
train_features = load(open("./encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))